In [1]:
import matplotlib.pyplot as plt
import numpy as np
from   tensorflow.keras import regularizers
from   tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from   tensorflow.keras.datasets import fashion_mnist
from   tensorflow.keras.layers import Dense, Input
from   tensorflow.keras.models import Model

%matplotlib inline

In [2]:
(X_train, _), (X_test, _) = fashion_mnist.load_data()
X_train = X_train.astype('float32') / 255.
X_test = X_test.astype('float32') / 255.

In [3]:
X_train.shape, X_test.shape

((60000, 28, 28), (10000, 28, 28))

In [4]:
# Flatten
X_train = X_train.reshape((X_train.shape[0], np.prod(X_train.shape[1:])))
X_test = X_test.reshape((X_test.shape[0], np.prod(X_test.shape[1:])))

In [5]:
X_train.shape, X_test.shape

((60000, 784), (10000, 784))

In [6]:
image_dim = X_train.shape[1]
encoding_dim = 32

In [8]:
input_image = Input(shape=(image_dim,))
encoded_image = Dense(
    encoding_dim, 
    activation='relu', 
    activity_regularizer=regularizers.l1(10e-5)
)(input_image)
encoder = Model(input_image, encoded_image)
decoded_image = Dense(image_dim, activation='sigmoid')(encoded_image)
autoencoder = Model(input_image, decoded_image)

In [10]:
encoded_input = Input(shape=(encoding_dim,))
decoder_layer = autoencoder.layers[-1]
decoder = Model(encoded_input, decoder_layer(encoded_input))

In [11]:
autoencoder.compile(optimizer='adadelta', loss='binary_crossentropy')

In [14]:
checkpointer1 = ModelCheckpoint(
    filepath='modWBest.hdf5', verbose=2, save_best_only=True)

In [15]:
checkpointer2 = EarlyStopping(monitor='val_loss', 
                              min_delta=0.0005, 
                              patience=2, 
                              verbose=2, 
                              mode='auto')